In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt

from sen3r.sen3r import Core
from sen3r.nc_engine import NcEngine
from sen3r.commons import Utils
from pathlib import Path

## Running SEN3R with a single WFR image

In [2]:
# TESTING SINGLE IMAGE PROCESSING
args = {}

args['input'] = Path('D:/S3/sen3r_wfr_test/S3A_OL_2_WFR____20160522T134229_20160522T134429_20171031T210832_0119_004_238______MR1_R_NT_002.SEN3')
args['out'] = Path('D:/sen3r_out/')
args['roi'] = Path('D:/A1_JM/areas/paper_areas/14100000_manacapuru.geojson')
args['product'] = 'wfr'
args

{'input': WindowsPath('D:/S3/sen3r_wfr_test/S3A_OL_2_WFR____20160522T134229_20160522T134429_20171031T210832_0119_004_238______MR1_R_NT_002.SEN3'),
 'out': WindowsPath('D:/sen3r_out'),
 'roi': WindowsPath('D:/A1_JM/areas/paper_areas/14100000_manacapuru.geojson'),
 'product': 'wfr'}

In [3]:
s3r = Core(args)

In [139]:
%%time
band_data, img_data, doneList = s3r.build_single_file()

INFO:D:\sen3r_out\sen3r_20211101T155557.log:Searching for WFR file inside: D:\S3\sen3r_wfr_test\S3A_OL_2_WFR____20160522T134229_20160522T134429_20171031T210832_0119_004_238______MR1_R_NT_002.SEN3
INFO:D:\sen3r_out\sen3r_20211101T155557.log:Generating ancillary data folder: D:\sen3r_out\CSV_N1
INFO:D:\sen3r_out\sen3r_20211101T155557.log:Attempting to extract geometries from: D:\A1_JM\areas\paper_areas\14100000_manacapuru.geojson
INFO:D:\sen3r_out\sen3r_20211101T155557.log:4428 - Initializing geometries for: S3A_OL_2_WFR____20160522T134229_20160522T134429_20171031T210832_0119_004_238______MR1_R_NT_002
INFO:D:\sen3r_out\sen3r_20211101T155557.log:Converting the polygon coordinates into a matrix x,y poly...
INFO:D:\sen3r_out\sen3r_20211101T155557.log:Saving DF at : D:\sen3r_out\CSV_N1\S3A_OL_2_WFR____20160522T134229_20160522T134429_20171031T210832_0119_004_238______MR1_R_NT_002.csv


Wall time: 17.5 s


## For a folder of WFR images

In [22]:
mt_args = {}

mt_args['input'] = Path('D:/S3/sen3r_wfr_test/')
mt_args['out'] = Path('D:/sen3r_out/')
mt_args['roi'] = Path('D:/A1_JM/areas/paper_areas/14100000_manacapuru.geojson')
mt_args['product'] = 'wfr'
mt_args

s3r = Core(mt_args)

In [ ]:
%%time
doneList = s3r.build_intermediary_files()

In [24]:
doneList

['D:\\amigos\\laura_cuba\\sen3r_out\\AUX_DATA\\S3B_OL_2_WFR____20211001T151242_20211001T151542_20211003T044606_0179_057_296_2520_MAR_O_NT_003.csv']